In [1]:
import os
import cv2
import gc
import time
from keras.applications.imagenet_utils import preprocess_input
import numpy as np  
np.set_printoptions(threshold=np.inf)
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

from keras import layers, activations
from keras.models import Model
from keras.layers import Input,Dense,Flatten,Dropout,ZeroPadding2D,BatchNormalization,Activation,Add,Dot,AveragePooling2D,Lambda
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D,MaxPooling2D  

Using TensorFlow backend.


In [2]:
def read_image(image_name):
    im=cv2.imread(image_name)
    im=cv2.resize(im, (224, 224))
    return im

#数据扩充
def img_Rotation(img,angel):
    if(0 == angel):
        dst = img
    else:
        rows,cols=img.shape[:2]
        #angel度旋转
        M=cv2.getRotationMatrix2D((cols/2,rows/2),angel,1)
        dst=cv2.warpAffine(img,M,(cols,rows))
    
    return dst

In [3]:
dataset = 'PatternNet'
Extension='.jpg'

base_path='./Dataset/' + dataset

All_Labels = os.listdir(os.path.join(base_path, 'Train'))
num_classes = len(All_Labels)
images_Train = []
labels_Train = []
images_Val = []
labels_Val = []

for i in range(0, num_classes):
    file_dir = os.path.join(base_path, 'Train', All_Labels[i])
    file_names = os.listdir(file_dir)
    for file_name in file_names:
        if(file_name.endswith(Extension)):
            finalFileName = os.path.join(file_dir, file_name)
            Label = np.linspace(0, 0, num_classes, dtype='int32')
            Label[i] = 1
            original_img = read_image(finalFileName)
            flipped_img = cv2.flip(original_img, 0)
            
            for j in [0, 45, 90, 135, 180, 225, 270, 315]: 
                images_Train.append(img_Rotation(original_img, j))
                labels_Train.append(Label)               
                images_Train.append(img_Rotation(flipped_img, j))
                labels_Train.append(Label)               
                
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + "Get %d Train Images" %(len(images_Train)))

2021-07-18 10:26:24Get 16000 Train Images


In [4]:
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + " np.array")
X_Train = np.array(images_Train, dtype='float32')
Y_Train = np.array(labels_Train)
del images_Train
gc.collect()

print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + " preprocess")
X_Train = preprocess_input(X_Train)

gc.collect()
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + " Done!")

2021-07-18 10:26:24 np.array
2021-07-18 10:27:39 preprocess
2021-07-18 10:27:50 Done!


In [5]:
ModelName = "Gabor-CA-ResNet50"
batch_size = 20
epochs = 50  
num_classes = Y_Train.shape[1]
#num_classes = 19

In [6]:
import cv2
def Gabor(shape, dtype=None):    
    ksize = (7, 7)
    # 核尺寸
    #sigmas = [1] # [2, 4]
    # 角度
    #thetas = np.linspace(0, 2*np.pi, 8, endpoint=False) # np.linspace(0, np.pi, 4, endpoint=False)
    # 波长(间隔)
    #lambdas = np.linspace(2, 3, 6) # [8, 16, 32, 64]
    # 高度(越小，核函数图像会越高)
    #gammas = [1] # np.linspace(1, 0, 2, endpoint=False)
    # 中轴
    #psis = [0, 2*np.pi]
    
    gabors = []
    
    for i in range(0,int(64/4)):    
#     size, sigma, theta, lambda, gamma aspect ratio                 
        gf = cv2.getGaborKernel(ksize=ksize, sigma=1, theta=0, lambd=2, gamma=1, psi=0, ktype=cv2.CV_32F)
        gabors.append(gf)             
        gf = cv2.getGaborKernel(ksize=ksize, sigma=1, theta=np.pi/2, lambd=2, gamma=1, psi=0, ktype=cv2.CV_32F)
        gabors.append(gf)             
        gf = cv2.getGaborKernel(ksize=ksize, sigma=1, theta=np.pi/4, lambd=2, gamma=1, psi=0, ktype=cv2.CV_32F)
        gabors.append(gf)             
        gf = cv2.getGaborKernel(ksize=ksize, sigma=1, theta=np.pi/4*3, lambd=2, gamma=1, psi=0, ktype=cv2.CV_32F)
        gabors.append(gf)
    stacked_list = np.array([gabors])
    stacked_list = np.einsum('hijk->jkhi', stacked_list)
    
    b = K.constant(stacked_list, dtype='float32')
    F_0 = Lambda(lambda x: K.cast(x, dtype='float32'))(b)
    return F_0

In [7]:
bn_params = {
    'epsilon': 9.999999747378752e-06,
}

def residual_block(input_tensor, filters, reduction=16, strides=1, **kwargs):
    x = input_tensor
    residual = input_tensor

    # bottleneck
    x = Conv2D(filters // 4, (1, 1), kernel_initializer='he_uniform', strides=strides, use_bias=False)(x)
    x = BatchNormalization(**bn_params)(x)
    x = Activation('relu')(x)

    x = ZeroPadding2D(1)(x)
    x = Conv2D(filters // 4, (3, 3), kernel_initializer='he_uniform', use_bias=False)(x)
    x = BatchNormalization(**bn_params)(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, (1, 1), kernel_initializer='he_uniform', use_bias=False)(x)
    x = BatchNormalization(**bn_params)(x)

    #  if number of filters or spatial dimensions changed
    #  make same manipulations with residual connection
    x_channels = K.int_shape(x)[-1]
    r_channels = K.int_shape(residual)[-1]

    if strides != 1 or x_channels != r_channels:

        residual = Conv2D(x_channels, (1, 1), strides=strides, kernel_initializer='he_uniform', use_bias=False)(residual)
        residual = BatchNormalization(**bn_params)(residual)

    # apply attention module
    x = ChannelSE(x, reduction=reduction)

    # add residual connection
    x = Add()([x, residual])

    x = Activation('relu')(x)

    return x

def ChannelSE(input_tensor, reduction=16):  
    channels = K.int_shape(input_tensor)[-1]
    x = Lambda(lambda a: K.mean(a, axis=[1, 2], keepdims=True))(input_tensor)
    x = Conv2D(channels // reduction, (1, 1), kernel_initializer='he_uniform', activation=activations.relu)(x)
    x = Conv2D(channels, (1, 1), kernel_initializer='he_uniform', activation=activations.hard_sigmoid)(x)
    return layers.Multiply()([input_tensor,x])    #给通道加权重
    
from keras.optimizers import SGD  
def GaborCAResNet(shape):
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
        
    img_input = Input(shape=shape)  

    x = ZeroPadding2D(3)(img_input)
    x = Conv2D(64, (7, 7), strides=2, use_bias=False, kernel_initializer='he_uniform')(x)
    x = BatchNormalization(**bn_params)(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(3)(x)
    x = Conv2D(64, (7, 7), strides=1, use_bias=False, kernel_initializer=Gabor, name='Gabor')(x)
    #x = Conv2D(64, (7, 7), strides=1, use_bias=False, name='Gabor')(x)
    x = BatchNormalization(**bn_params, name='Gabor-Batch')(x)
    x = Activation('relu', name='Gabor-relu')(x)

    x = ZeroPadding2D(1)(x)
    x = MaxPooling2D((3, 3), strides=2)(x)
    
    # body of resnet
    filters = 128
    for i, stage in enumerate([3, 4, 6, 3]):

        # increase number of filters with each stage
        filters *= 2
        reduction = 16

        for j in range(stage):
            # decrease spatial dimensions for each stage (except first, because we have maxpool before)
            if i == 0 and j == 0:
                x = residual_block(x, filters, reduction=reduction, strides=1, is_first=True)
            elif i != 0 and j == 0:
                x = residual_block(x, filters, reduction=reduction, strides=2)
            else:
                x = residual_block(x, filters, reduction=reduction, strides=1)
    
   

    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dense(num_classes, activation='softmax', name='output_AID')(x)  

    model = Model(inputs=img_input,outputs=x, name=ModelName)  
    sgd = SGD(decay=0.0001,momentum=0.9)  
    model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])  
    
    return model

In [13]:
K.clear_session()
tf.reset_default_graph()

In [8]:
#model = GaborCAResNet(X_Train.shape[1:])
model = GaborCAResNet([224,224,3])
model.load_weights('./models/2021-06-25/PatternNet/Gabor-CA-ResNet50.h5', by_name=True)
model.summary()
model.layers[6].trainable = False

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "Gabor-CA-ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         conv2d_1[0][0]                   


In [9]:
history = model.fit(X_Train, Y_Train, batch_size=batch_size, epochs=epochs, shuffle=True)

D:\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/50
16000/16000 [==============================] - 153s 10ms/step - loss: 0.2459 - accuracy: 0.9284
Epoch 2/50
16000/16000 [==============================] - 141s 9ms/step - loss: 0.0130 - accuracy: 0.9971
Epoch 3/50
16000/16000 [==============================] - 140s 9ms/step - loss: 0.0076 - accuracy: 0.9978
Epoch 4/50
16000/16000 [==============================] - 141s 9ms/step - loss: 0.0044 - accuracy: 0.9987
Epoch 5/50
16000/16000 [==============================] - 141s 9ms/step - loss: 0.0034 - accuracy: 0.9991
Epoch 6/50
16000/16000 [==============================] - 141s 9ms/step - loss: 7.5020e-04 - accuracy: 1.0000
Epoch 7/50
16000/16000 [==============================] - 142s 9ms/step - loss: 4.5413e-04 - accuracy: 1.0000
Epoch 8/50
16000/16000 [==============================] - 142s 9ms/step - loss: 5.2472e-04 - accuracy: 1.0000
Epoch 9/50
16000/16000 [====================

In [10]:
import time
save_folder = 'models/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + dataset
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# serialize model to JSON
#import pickle
model_json = model.to_json()
with open(os.path.join(save_folder, ModelName + ".json"), "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save(os.path.join(save_folder, ModelName + ".h5"))
print("Saved Model to disk")

Saved Model to disk


In [11]:
import os
import time
import cv2
from keras.applications.imagenet_utils import preprocess_input

base_path='./Dataset/' + dataset
Extension='.jpg'
All_Labels = os.listdir(os.path.join(base_path, 'Test'))
images = []
labels = []
outputFileName = []

save_folder = 'FeatureMap/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + ModelName + '/' + dataset  +'/'

for i in range(0, num_classes):
    file_dir = os.path.join(base_path, 'Test', All_Labels[i])
    file_names = os.listdir(file_dir)
    for file_name in file_names:
        if(file_name.endswith(Extension)):
            finalFileName = os.path.join(file_dir, file_name)
            Label = np.linspace(0, 0, num_classes, dtype='int32')
            Label[i] = 1    
            images.append(read_image(finalFileName))
            labels.append(Label)
            outputFileName.append(os.path.join(save_folder, All_Labels[i], file_name).replace(Extension, ".txt"))

print("Get %d Test Images" %(len(images)))
output = np.array(images, dtype="float32")
output = preprocess_input(output)

Get 9000 Test Images


In [12]:
import os
import time
from keras.models import Model  
OutPutLayer = Model(inputs=model.input, outputs=model.get_layer('avg_pool').output)
print("Saved FeatureMap to disk...")
OutputFeatures=[]
for i in range(0, len(output)):
    p = OutPutLayer.predict(output[i : i + 1])
    out=np.reshape(p,p.shape[1])
    OutputFeatures.append(out)
    print("\r当前输出：%d" %(i + 1), end= " ")

OutputFeatures = np.array(OutputFeatures, dtype="float") 
save_folder = 'FeatureMap/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + ModelName + '/' + dataset  +'/'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
np.savetxt(os.path.join(save_folder, dataset + '-Test-2048D.txt'), OutputFeatures)
print("\n保存完成！")
print(OutputFeatures.shape)

Saved FeatureMap to disk...
当前输出：9000                              
保存完成！
(9000, 2048)


import os
import time
from keras.models import Model  
OutPutLayer = Model(inputs=model.input, outputs=model.get_layer('avg_pool').output)
print("Saved FeatureMap to disk...")
OutputFeatures=[]
for i in range(0, len(X_Train)):
    p = OutPutLayer.predict(X_Train[i : i + 1])
    out=np.reshape(p,p.shape[1])
    OutputFeatures.append(out)
    print("\r当前输出：%d" %(i + 1), end= " ")

OutputFeatures = np.array(OutputFeatures, dtype="float") 
save_folder = 'FeatureMap/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + ModelName + '/' + dataset  +'/'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
np.savetxt(os.path.join(save_folder, dataset + '-Train-2048D.txt'), OutputFeatures)
print("\n保存完成！")
print(OutputFeatures.shape)

In [10]:
import os
import time
import cv2
from keras.applications.imagenet_utils import preprocess_input

base_path='./Dataset/' + dataset
Extension='.jpg'
All_Labels = os.listdir(os.path.join(base_path, 'Train'))
images = []
labels = []
outputFileName = []

save_folder = 'FeatureMap/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + ModelName + '/' + dataset  +'/'

for i in range(0, num_classes):
    file_dir = os.path.join(base_path, 'Train', All_Labels[i])
    file_names = os.listdir(file_dir)
    for file_name in file_names:
        if(file_name.endswith(Extension)):
            finalFileName = os.path.join(file_dir, file_name)
            Label = np.linspace(0, 0, num_classes, dtype='int32')
            Label[i] = 1    
            images.append(read_image(finalFileName))
            labels.append(Label)
            outputFileName.append(os.path.join(save_folder, All_Labels[i], file_name).replace(Extension, ".txt"))

print("Get %d Test Images" %(len(images)))
output = np.array(images, dtype="float32")
output = preprocess_input(output)

Get 100 Test Images


In [15]:
import os
import time
from keras.models import Model  
OutPutLayer = Model(inputs=model.input, outputs=model.get_layer('avg_pool').output)
print("Saved FeatureMap to disk...")
OutputFeatures=[]
for i in range(0, len(output)):
    p = OutPutLayer.predict(output[i : i + 1])
    out=np.reshape(p,p.shape[1])
    OutputFeatures.append(out)
    print("\r当前输出：%d" %(i + 1), end= " ")

OutputFeatures = np.array(OutputFeatures, dtype="float") 
save_folder = 'FeatureMap/' + time.strftime("%Y-%m-%d", time.localtime()) + '/' + ModelName + '/' + dataset  +'/'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
np.savetxt(os.path.join(save_folder, dataset + '-Train-2048D.txt'), OutputFeatures)
print("\n保存完成！")
print(OutputFeatures.shape)

Saved FeatureMap to disk...
当前输出：6080         
保存完成！
(6080, 2048)
